In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import datetime
from dotenv import load_dotenv
load_dotenv();

### Generate a random Hamiltonian instance

* In this example, we generate a random Sherrington-Kirkpatrick Hamiltonian.
* In general, we can generate other classes, or read the instances from file.
* We can also solve the Hamiltonian classicaly, to get the ground state energy and the highest energy. (this, of course, should be done offline, before running the QAOA)



In [ ]:
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.data_analysis.data_handling import (COEFFICIENTS_TYPE,
                                                 COEFFICIENTS_DISTRIBUTION,
                                                 CoefficientsDistributionSpecifier,
                                                 HAMILTONIAN_MODELS)

number_of_qubits = 10
seed_cost_hamiltonian = 1

coefficients_type = COEFFICIENTS_TYPE.CONTINUOUS
coefficients_distribution = COEFFICIENTS_DISTRIBUTION.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

# We generate a Hamiltonian instance. In this case it's a random Sherrington-Kirkpatrick Hamiltonian
hamiltonian_model = HAMILTONIAN_MODELS.SherringtonKirkpatrick
localities = (2,)
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True)

print("Class description (cost):", cost_hamiltonian.hamiltonian_class_description)
print("Instance description (cost):", cost_hamiltonian.hamiltonian_instance_description)

if cost_hamiltonian.lowest_energy is None:
    print("SOLVING THE HAMILTONIAN CLASSICALLY")
    # if we wish, we can solve the Hamiltonian classically
    cost_hamiltonian.solve_hamiltonian(both_directions=True)

ground_state_energy = cost_hamiltonian.ground_state_energy
highest_energy = cost_hamiltonian.highest_energy

### Wrapper for QAOA Sampler
* In the context of QAOA, we can wrap the simulator a bit more so it's a part of more abstract framework useful in further optimization.
* The "QAOARunnerSampler" is an abstract class that can be used to run the QAOA with different backends (here we will be interested mostly in qiskit though).
* The optimization is a bit more general -- a) we can optimize over ansatz for which phase is not the same as cost; b) we can optimize over multiple representations of the Hamiltonian (e.g., permutations or bitflip gauges); note that b) requires classical optimizer that can actually support it.


In [ ]:
from quapopt.optimization.QAOA.implementation.QAOARunnerSampler import QAOARunnerSampler
from quapopt.data_analysis.data_handling import LoggingLevel

#Specify some kwargs for logging (this is generally not required, but here we want to log the results in a specific folder)
experiment_set_name = 'QAOAOptimization'
experiment_set_id = 'TestRuns1'

logging_level = LoggingLevel.DETAILED


#The logger will create subfolders specified here
logger_kwargs_main = {'experiment_folders_hierarchy': ['SimulationResults',
                                                       'TestingQAOAOptimization',
                                                       f'TestRuns_{datetime.datetime.today().strftime("%Y-%m-%d")}_{np.random.randint(1000)}'],
                      'experiment_set_name':experiment_set_name, #Used to group the experiments
                      'experiment_set_id':experiment_set_id, #Used to group the experiments
                      }

print(logger_kwargs_main)



In [ ]:
sampler_backend = 'Qiskit'

In [ ]:
logger_kwargs_sampler = {**logger_kwargs_main,
                         **{'experiment_instance_id': f"IdealSampling{sampler_backend.upper()}"}}



numpy_rng_sampling = np.random.default_rng(seed=0)
#Set up the raw sampler with Hamiltonians as an input
qaoa_sampler = QAOARunnerSampler(
    hamiltonian_representations_cost=[cost_hamiltonian],
    hamiltonian_representations_phase=None,
    store_n_best_results=1,
    store_full_information_in_history=False,
    numpy_rng_sampling=numpy_rng_sampling,
    logger_kwargs=logger_kwargs_sampler,
    logging_level=logging_level
)

#Here we basically run the same thing as before but inside the class;
#so the ansatze are set up for possibly multiple cost Hamiltonians
if sampler_backend.lower() == 'qiskit':
    qaoa_sampler.initialize_backend_qiskit(qaoa_depth=1)
elif sampler_backend.lower() == 'qokit':
    #If we have GPU, we use qokit for demonstration because it's faster than qiskit
    qaoa_sampler.initialize_backend_qokit(qokit_backend='gpu')
else:
    raise NotImplementedError("Only qiskit and qokit are supported")

#print(qaoa_sampler._backends[0].ansatz_circuit)

### Wrapper for optimizers

* In general, we will want to use our QAOA samplers with some optimizers.
* We can run:
1. Optuna -- a hyperparameter optimization framework. It has some nice optimizers and visualization features, but it's not the fastest. Somehow they have huge overheads for long optimizations.
2. Custom grid sampling - we can run a simple grid sampling to get a feel for the landscape.
3. Scipy -- we can also use scipy's optimizers.




In [ ]:
from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner
from quapopt.optimization.parameter_setting.variational.scipy_tools.ScipyOptimizerWrapped import ScipyOptimizerWrapped

#we can specify the details of classical optimizer here.
classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[(-np.pi, np.pi)]*2,
                                                              #argument_names = ['Angles-0', 'Angles-1'],
                                            optimizer_name='COBYLA',
                                            optimizer_kwargs=None,
                                            basinhopping=True,
                                            basinhopping_kwargs={'niter':1},
                                            starting_point= [0.05]*2
                                            )



qaoa_optimizer = QAOAOptimizationRunner(qaoa_runner=qaoa_sampler)

In [ ]:
#Number of objective function calls
number_of_function_calls = 200
#Number of measurements to estimate the expectation values
number_of_samples = 10**3

best_results_noiseless, optimization_results_noiseless = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                                                            number_of_function_calls=number_of_function_calls,
                                                                                         number_of_samples=number_of_samples,
                                                                                         classical_optimizer=classical_optimizer,
                                                                                         optimizer_seed=42,
                                                                                         #Note that we can add readout noise here
                                                                                         measurement_noise=None,
                                                                                         verbosity=1,
                                                                                         show_progress_bar=True)



In [ ]:
#note: "best" result here means the one containing the best BITSTRING, not necessary the best expected value

best_result_noiseless = best_results_noiseless[0]
best_energy_bts_noiseless = best_result_noiseless[0]
best_bts_noiseless = best_result_noiseless[1][0]
best_metadata_noiseless = best_result_noiseless[1][-1]

print("BEST ENERGY (BTS, NOISELESS):", best_energy_bts_noiseless)
print("BEST BTS (NOISELESS):", best_bts_noiseless)
print('BEST MEAN ENERGY (NOISELESS):', optimization_results_noiseless.best_value)
best_metadata_noiseless.to_dataframe_main()

### Add measurement noise

* We can add classical measurement noise to the simulation. Here we will emulate amplitude damping.

In [ ]:
from quapopt.circuits.noise.simulation.ClassicalMeasurementNoiseSampler import ClassicalMeasurementNoiseSampler, \
    MeasurementNoiseType

# Fully asymmetric noise -- equivalent to amplitude damping at the end of the circuit
p_01 = 0.5
p_10 = None
CMNS = ClassicalMeasurementNoiseSampler(noise_type=MeasurementNoiseType.TP_1q_identical,
                                        noise_description={'p_01': p_01,
                                                           'p_10': p_10})

logger_kwargs_sampler_noisy = {**logger_kwargs_main,
                               **{'experiment_instance_id': f"NoisySampling{sampler_backend.upper()}"}}
qaoa_optimizer.qaoa_runner.reinitialize_logger(**logger_kwargs_sampler_noisy)

best_results_noisy, optimization_results_noisy = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                                                 number_of_function_calls=number_of_function_calls,
                                                                                 number_of_samples=number_of_samples,
                                                                                 classical_optimizer=classical_optimizer,
                                                                                 optimizer_seed=42,
                                                                                 #Note that are adding noise here
                                                                                 measurement_noise=CMNS,
                                                                                 verbosity=1,
                                                                                 show_progress_bar=True
                                                                                 )


In [ ]:
best_result_noisy = best_results_noisy[0]

best_energy_bts_noisy = best_result_noisy[0]
best_bts_noisy = best_result_noisy[1][0]
best_metadata_noisy = best_result_noisy[1][-1]

print("BEST ENERGY (BTS, NOISELESS):", best_energy_bts_noiseless)
print("BEST BTS (NOISELESS):", best_bts_noiseless)
print('BEST MEAN ENERGY (NOISELESS):', optimization_results_noiseless.best_value)
print("____________")
print("BEST ENERGY (BTS, NOISY):", best_energy_bts_noisy)
print("BEST BTS:", best_bts_noisy)
print('BEST MEAN ENERGY (NOISY):', optimization_results_noisy.best_value)

### Reading data

* In a moment, we will want to visualize data.
* We could take the data directly from the output of optimization, but here we will read it from database to test the logging:
1. We need to set up the logger for the results. (the optimizer does it automatically, but we are pretending we ran the optimization separately and now just reading the results).
2. The input requires: the cost Hamiltonian, and potentially the folder hierarchy and the main table name prefix so it can identify what data we want to read.
3. We can then read the data from the database and visualize it.

In [ ]:
from quapopt.optimization.QAOA import QAOAResultsLogger
from quapopt.data_analysis.data_handling import (STANDARD_NAMES_DATA_TYPES as SNDT,
                                                 STANDARD_NAMES_VARIABLES as SNV)

results_logger_sampler = QAOAResultsLogger(**{'cost_hamiltonian': cost_hamiltonian},
                                           **logger_kwargs_sampler)

df_noiseless = results_logger_sampler.read_results(data_type=SNDT.OptimizationOverview)
df_noiseless_direct = optimization_results_noiseless.trials_dataframe

results_logger_sampler_noisy = QAOAResultsLogger(**{'cost_hamiltonian': cost_hamiltonian},
                                                 **logger_kwargs_sampler_noisy)
df_noisy = results_logger_sampler_noisy.read_results(data_type=SNDT.OptimizationOverview)
df_noisy_direct = optimization_results_noisy.trials_dataframe


In [ ]:
df_noiseless

In [ ]:
df_noiseless_direct

In [ ]:
df_noisy

In [ ]:
df_noisy_direct

### Visualization

* After running optimization, we can visualize it using various methods with plotly.


In [ ]:
import plotly
from quapopt.data_analysis.visualization import optimization_visualization as opt_vis

plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

x_name = f"{SNV.Angles.id_long}-0"
y_name = f"{SNV.Angles.id_long}-1"
fom_name = f"{SNV.EnergyMean.id_long}"

In [ ]:
trajectory_fig = opt_vis.plot_multiple_heatmaps(dataframes=[df_noiseless, df_noisy],
                                                x_name=x_name,
                                                y_name=y_name,
                                                fom_name=fom_name,
                                                bounds_x=(-np.pi, np.pi),
                                                bounds_y=(-np.pi, np.pi),
                                                add_trajectories=True,
                                                titles=['Ideal', 'Noisy'],
                                                global_normalization=True,
                                                colormap_name='Viridis',
                                                minimization=True)

plotly.offline.plot(trajectory_fig,
                    filename='../temp/trajectories_plot.html')
trajectory_fig.show()

### Grid sampling
* We can also run a simple grid sampling to get a feel for the landscape and to compare it with the optimization results.
* Here we run bruteforce grid sampling that is not scalable for large dimensions
* We will use it to plot the background grid of points for the trajectories of the TPE optimizer.
* Note that this works only for p=1, otherwise it's hard to visualize

In [ ]:
from quapopt.optimization.parameter_setting.non_adaptive_optimization.SimpleGridOptimizer import SimpleGridOptimizer
from quapopt.optimization.parameter_setting import ParametersBoundType as PBT
from quapopt.optimization.QAOA.simulation.QAOARunnerExpValues import QAOARunnerExpValues

grid_size_total = 10 ** 4

grid_sampler = SimpleGridOptimizer(parameter_bounds=[(PBT.RANGE, (-np.pi, np.pi)),
                                                     (PBT.RANGE, (-np.pi, np.pi))
                                                     ],
                                   max_trials=grid_size_total)

logger_kwargs_grid = {**logger_kwargs_main,
                      **{'table_name_prefix': 'IdealQokitGrid'}}

#runner that calculates expected values for p=1 QAOA
qaoa_exp_values_simulator = QAOARunnerExpValues(
                                                  hamiltonian_representations_cost=[cost_hamiltonian],
                                                  store_full_information_in_history=False,
                                                  simulator_name=None,
                                                  logger_kwargs=logger_kwargs_grid,
                                                  logging_level=None)


qaoa_optimizer_exp_values = QAOAOptimizationRunner(qaoa_runner=qaoa_exp_values_simulator)


In [ ]:
#noiseless grid
best_result_grid_noiseless, opt_results_grid_noiseless = qaoa_optimizer_exp_values.run_optimization(
    qaoa_depth=1,
    number_of_function_calls=grid_size_total,
    classical_optimizer=grid_sampler,
    measurement_noise=None,
    memory_intensive=True,
    show_progress_bar=True,
    verbosity=1,
    # analytical_betas=True
)


In [ ]:
#noisy grid
best_result_grid_noisy, opt_results_grid_noisy = qaoa_optimizer_exp_values.run_optimization(qaoa_depth=1,
                                                                                            number_of_function_calls=grid_size_total,
                                                                                            classical_optimizer=grid_sampler,
                                                                                            memory_intensive=True,
                                                                                            verbosity=1,
                                                                                            measurement_noise=CMNS,
                                                                                            show_progress_bar=True
                                                                                            )

In [ ]:
#this can made it a bit slower than just opening in browser, but it allows to show the plots in Jupyter cells
show_plots_in_jupyter_cell = True

#TODO(FBM): this should be unnecessary, refactor
names_map = {f"ARG-0": f"{SNV.Angles.id_long}-0",
             f"ARG-1": f"{SNV.Angles.id_long}-1",
             f"FunctionValue": f"{SNV.EnergyMean.id_long}"}

df_noiseless_grid = opt_results_grid_noiseless.trials_dataframe
df_noiseless_grid.rename(columns=names_map, inplace=True)

df_noisy_grid = opt_results_grid_noisy.trials_dataframe
df_noisy_grid.rename(columns=names_map, inplace=True)

for plot_in_3d in [False,True]:
    heatmap_grid_noiseless, scatter_grid_noiseless = opt_vis.get_interpolated_heatmap(df=df_noiseless_grid,
                                                                                      x_name=x_name,
                                                                                      y_name=y_name,
                                                                                      fom_name=fom_name,
                                                                                      bounds_x=(-np.pi, np.pi),
                                                                                      bounds_y=(-np.pi, np.pi),
                                                                                      in_3d=plot_in_3d
                                                                                      )
    heatmap_grid_noisy, scatter_grid_noisy = opt_vis.get_interpolated_heatmap(df=df_noisy_grid,
                                                                              x_name=x_name,
                                                                              y_name=y_name,
                                                                              fom_name=fom_name,
                                                                              bounds_x=(-np.pi, np.pi),
                                                                              bounds_y=(-np.pi, np.pi),
                                                                              in_3d=plot_in_3d
                                                                              )

    trajectory_fig2 = opt_vis.plot_multiple_heatmaps(dataframes=[df_noiseless, df_noisy],
                                                     x_name=x_name,
                                                     y_name=y_name,
                                                     fom_name=fom_name,
                                                     bounds_x=(-np.pi, np.pi),
                                                     bounds_y=(-np.pi, np.pi),
                                                     heatmaps_input=[heatmap_grid_noiseless, heatmap_grid_noisy],
                                                     scatter_inputs=[scatter_grid_noiseless, scatter_grid_noisy],
                                                     add_trajectories=True,
                                                     titles=['Ideal', 'Noisy'],
                                                     suffixes=['(Ideal)', '(Noisy)'],
                                                     global_normalization=False,
                                                     colormap_name='Viridis',
                                                     minimization=True,
                                                     in_3d=plot_in_3d)

    plotly.offline.plot(trajectory_fig2,
                        filename=f'../temp/trajectories_grid_plot_3d-{plot_in_3d}.html')
    if not show_plots_in_jupyter_cell:
        continue
    trajectory_fig2.show()
